# Importing the required Keras dependancies

In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
import keras
from keras import regularizers, optimizers
from keras.layers import Conv2D,Input,Dense,MaxPooling2D,BatchNormalization,ZeroPadding2D,Flatten,Dropout
from keras.models import Model
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, CSVLogger,EarlyStopping,ModelCheckpoint

Using TensorFlow backend.


# Other dependancies

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from numpy.random import permutation

In [3]:
def LeNet_Model():
    # creating a sequential model
    model = Sequential()
    
    # adding first set of CONV -> RELU -> POOL
    model.add(Convolution2D(20, 5, 5, border_mode="same",input_shape=(60, 60,3))) # 3,60,60 tha pehle
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # adding second set of CONV -> RELU -> POOL
    model.add(Convolution2D(50, 5, 5, border_mode="same"))
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2, 2),dim_ordering='th')) # specifying the order of dimensions

    # adding set of FC -> RELU layers
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
 
    # adding softmax classifier
    model.add(Dense(7))          # 7 categories
    model.add(Dropout(0.2))
    model.add(Activation("softmax"))
        
    return model

# Creating the model

In [4]:
model= LeNet_Model()

C:\Users\shubham saksham\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (5, 5), input_shape=(60, 60, 3..., padding="same")`
  
C:\Users\shubham saksham\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (5, 5), padding="same")`
  if sys.path[0] == '':
C:\Users\shubham saksham\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first")`
  from ipykernel import kernelapp as app


# Creating a summary of various layers and total parameters

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 20)        1520      
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60, 20)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 60, 60, 20)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 50)        25050     
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 30, 50)        0         
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 50)        0         
__________

# Loading the train images and labels

In [6]:
train_images = np.load('train_images_lenet.npy')
train_labels = np.load('train_labels_lenet.npy')

In [7]:
train_images.shape # 6359 images of 60x60 resolution and 3 channels

(6359, 60, 60, 3)

In [8]:
train_labels.shape

(6359,)

In [9]:
np.unique(train_labels)
np.bincount(train_labels)

array([ 591,  592,  590,   24,  592,  372, 3598], dtype=int64)

# Callbacks

In [10]:
# Reduce learning rate when a metric has stopped improving.
lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)

# Callback that streams epoch results to a csv file
csv_logger = CSVLogger('Lenet.csv')

# Stop training when a monitored quantity has stopped improving.
early_stopper = EarlyStopping(min_delta=0.001,patience=30)

# Save the model after every epoch.
model_checkpoint = ModelCheckpoint('Lenet.hdf5',monitor = 'val_loss', verbose = 1,save_best_only=True)

In [11]:
# Test pretrained model
train_images = np.array(train_images)
train_labels = np.array(train_labels)
mean = np.mean(train_images,axis=(0,1,2,3))
std = np.std(train_images,axis=(0,1,2,3))
train_images = (train_images-mean)/(std+1e-7)
num_classes = 7
train_labels = np_utils.to_categorical(train_labels,num_classes)

In [12]:
train_images.shape, train_labels.shape

((6359, 60, 60, 3), (6359, 7))

# Randomly distributing train_images to make train_images and val_images

In [13]:
perm = permutation(len(train_images))
train_images = train_images[perm]
train_labels = train_labels[perm]
new_train = train_images[1:4800]
new_labels = train_labels[1:4800]
val_images= train_images[4800:]
val_labels = train_labels[4800:]

In [14]:
new_train.shape,val_images.shape

((4799, 60, 60, 3), (1559, 60, 60, 3))

# Compiling the model

In [15]:
model.compile(loss='categorical_crossentropy',
        optimizer="Adam",
        metrics=['accuracy'])

# Preprocessing and data augmentation

In [16]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Fitting the train data

In [19]:
datagen.fit(new_train)
model.fit_generator(datagen.flow(new_train, new_labels, batch_size=12),
                        steps_per_epoch=new_train.shape[0] // 12,
                        epochs=30,
                        verbose=1,
                        validation_data=(val_images,val_labels))

Epoch 1/30
399/399 [==============================] - 123s 307ms/step - loss: 0.8658 - acc: 0.6816 - val_loss: 0.3305 - val_acc: 0.8942
Epoch 2/30
399/399 [==============================] - 121s 304ms/step - loss: 0.5086 - acc: 0.7744 - val_loss: 0.1888 - val_acc: 0.9416
Epoch 3/30
399/399 [==============================] - 122s 306ms/step - loss: 0.3764 - acc: 0.8183 - val_loss: 0.1186 - val_acc: 0.9660
Epoch 4/30
399/399 [==============================] - 121s 303ms/step - loss: 0.3479 - acc: 0.8279 - val_loss: 0.1644 - val_acc: 0.9416
Epoch 5/30
399/399 [==============================] - 121s 303ms/step - loss: 0.3341 - acc: 0.8294 - val_loss: 0.1734 - val_acc: 0.9480
Epoch 6/30
399/399 [==============================] - 120s 302ms/step - loss: 0.2879 - acc: 0.8525 - val_loss: 0.0855 - val_acc: 0.9692
Epoch 7/30
399/399 [==============================] - 122s 307ms/step - loss: 0.3068 - acc: 0.8423 - val_loss: 0.1244 - val_acc: 0.9609
Epoch 8/30
399/399 [============================

# Saving the model

In [20]:
model.save('Tomato_diseases.h5')

# Loading the saved model

In [22]:
from keras.models import load_model

loaded_model= load_model('Tomato_diseases.h5')
loaded_model

In [24]:
class_names = {0:'bacterial_spot',1:'Healthy',2:'late_blight',3:'leaf_mold',4:'septorial_leaf_spot',5:'mosaic_virus',
               6:'yelow_curved'}

# Testing the model with random images of leaves

In [30]:
input_img= cv2.imread('dataset\septoriaLeafSpot\sep3.jpg', 1)
if input_img is not None:
    img = cv2.resize(input_img, (60,60)).astype(np.float32)
            
else:
    print("image not loaded")
    
img = (img-mean)/(std+1e-7)
img = np.expand_dims(img, axis=0)  
out = loaded_model.predict(img) 
print(out)
print(np.argmax(out))
print(class_names[np.argmax(out)])
# Correct!

[[  5.37608969e-07   1.69465206e-11   4.41435226e-08   5.01497011e-10
    9.96732354e-01   3.26405396e-03   2.98724240e-06]]
4
septorial_leaf_spot


In [61]:
#input_img= cv2.imread('dataset\lateblight\l4.png', 1)

input_img= cv2.imread("dataset/"+'lateblight'+'/'+'l4.png', 1)

if input_img is not None:
    img = cv2.resize(input_img,(60,60)).astype(np.float32)
else:
    print("image not loaded")
    
img = (img-mean)/(std+1e-7)
img = np.expand_dims(img, axis=0)  
out = loaded_model.predict(img) 
print(out)
print(np.argmax(out))
print(class_names[np.argmax(out)])
# Correct

[[  2.92730761e-06   5.03858688e-09   9.99797761e-01   1.55134074e-07
    1.99062692e-04   2.72000374e-11   1.23168334e-07]]
2
late_blight


# Complete!